In [ ]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.3/spark-3.2.3-bin-hadoop2.7.tgz
!tar xf spark-3.2.3-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.3-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
import time

# This function creates SparkContext and StreamingContext
# Do not change this function
def initStreamingContext():
    try:
      ssc.end()
    except:
      pass
    finally:
      spark_conf = SparkConf()\
            .setAppName("YourTest")\
            .setMaster("local[*]")
      sc = SparkContext.getOrCreate(spark_conf)
      # Creating Streaming Context with batch window size of 1 second
      ssc = StreamingContext(sc, 1)
      return ssc

In [ ]:
ssc = initStreamingContext()

robotData = ssc.socketTextStream("datasci.cs.uwaterloo.ca", 4321)
robotData.pprint()
ssc.start()

time.sleep(5)
ssc.stop()

In [ ]:
ssc = initStreamingContext()
robotData = ssc.socketTextStream("datasci.cs.uwaterloo.ca", 4321)

result = robotData.map(lambda line: line.split(",")[:-1])\
                  .reduceByWindow(lambda x, y: x + y, None, 3,1)\
                  .map(lambda lst: min(lst))\
                  
result.pprint()
ssc.start()
# Let's wait for 10 seconds before we stop the program.
# Feel free to change this value but make sure you change it back to 10 before submission.
time.sleep(10)
ssc.stop()

In [ ]:
def ratio(lst):
  s = sum([i[1] for i in lst])

  ratio_lst = [(i[0], i[1]/s) for i in lst]
  return sorted(ratio_lst, key=lambda t: t[1],reverse=True) 

def println(rdd):
  lst = rdd.collect()[0]
  
  for t in lst:
    t = [str(i) for i in t]
    s = " ".join(t)
    print(s)

  print("----------")



ssc = initStreamingContext()
robotData = ssc.socketTextStream("datasci.cs.uwaterloo.ca", 4321)
#### Your Solution to Question 2 here
result = robotData.map(lambda line: (line.split(",")[-1],1))\
                  .reduceByKeyAndWindow(lambda x, y: x + y, None, 3,1)\
                  .map(lambda x: [x])\
                  .reduceByWindow(lambda x, y: x + y, None,1,1)\
                  .map(ratio)\

result.foreachRDD(println)

ssc.start()
# Let's wait for 10 seconds before we stop the program.
# Feel free to change this value but make sure you change it back to 10 before submission.
time.sleep(10)
ssc.stop()